In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
data=pd.read_csv('../input/titanic/train.csv')

In [13]:
## sns.catplot 사용법
## category를 나눠서 시각화하고 싶을 때 data,x,y,kind를 정해줄것.
sns.catplot(data=data,x='Embarked',y='Survived',kind='bar')

## Survived가 0,1로 되어있어있다.
## bar에 세로줄은 S인 것들에 0,1의 추정치와 불확실성을 보여준다.

In [14]:
g=sns.FacetGrid(data=data,col='Survived',row='Embarked')
## 똑같이 data,col,row 이런식으로 주고 현재 가로는 Survived,세로는 Embarked
g.map(sns.scatterplot,'Age','Fare','Sex')
g.add_legend()
## mapping 한다. sns.scatterplot을
## scatterplot은 x='Age',y='Fare',hue='Sex' 이 순서는 인자를 주지않고 sns.scatterplot순서대로

## 처음 간단하게만 볼때만 사용.

In [15]:
## 1. countplot 
## 특정 조건에 만족하는 행들의 수를 보여주는
## 데이터 균형 정도를 확인하는데 쓰면 좋을 것 같다. 
## data와 x만 필요하다. 

## 2. point,bar plot
## 위에서 보여준 차트로, estimate 추정할 때 사용.
## 보통 예측값이 카테고리 일 때 사용하면 좋을 것 같다.

In [16]:
## seaborn 특징.
## data : dataframe.
## x,y : 가로,세로에 변수 이름을 문자열로 넣는다.
## hue : hue가 있다면 hue는 data로 나눠주는 것이다. (안에서 나누기)
## col : col이 있다면 col은 figure로 나눠주는 것이다. (밖에서 나누기)

## 좀 꼬아져있는데 sns.boxplot은 ax가 가능하고, sns.catplot은 ax가 안된다. 
## facetgrid 때문인것 같다. 

## facetgrid
## matplotlib의 subplots 느낌. figure를 나눠주는 느낌.
## 그래서 catplot은 facetgrid를 return한다.
## 즉, g=sns.catplot(..) 후에 g.set_xlabels(..)로 덧붙인다.

In [17]:
## 이상치 제거 방법.
## 꽤 좋은 방법인것 같다. 

def detect_outliers(df,n,features):
    """
    Takes a dataframe df of features and returns a list of the indices
    corresponding to the observations containing more than n outliers according
    to the Tukey method.
    """
    outlier_indices = []
    
    # iterate over features(columns)
    for col in features:
        # 1st quartile (25%)
        Q1 = np.percentile(df[col], 25)
        # 3rd quartile (75%)
        Q3 = np.percentile(df[col],75)
        # Interquartile range (IQR)
        IQR = Q3 - Q1
        
        # outlier step
        outlier_step = 1.5 * IQR
        
        # Determine a list of indices of outliers for feature col
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step )].index
        # append the found outlier indices for col to the list of outlier indices 
        outlier_indices.extend(outlier_list_col)        
        
    # select observations containing more than 2 outliers
    outlier_indices = Counter(outlier_indices)        
    multiple_outliers = list( k for k, v in outlier_indices.items() if v > n )
    
    return multiple_outliers  

In [18]:
## 데이터 합치기 
## 합치는 것은 모든 데이터 안에서 스케일을 맞추기 위해서인데
## data leakage - https://www.kaggle.com/c/titanic/discussion/174969 사실 합치는건 좋은게 아니라고한다. 

In [19]:
## 결측값 처리 방법

## 특정 열의 결측값들인 행들을 가져와서

print(data['Age'].isna().sum())

index_NaN_age = list(data["Age"][data["Age"].isnull()].index)

## loop를 돌며 결측값이라면 주변 열의 데이터와 비슷한 데이터들만 모아서 중앙값으로 채운다.
## 다만 그런 데이터 프레임이 없을 때 단순히 전체 데이터의 중앙값으로 채운다. 

for i in index_NaN_age :
    age_med = data["Age"].median()
    age_pred = data["Age"][((data['SibSp'] == data.iloc[i]["SibSp"]) & (data['Parch'] == data.iloc[i]["Parch"]) & (data['Pclass'] == data.iloc[i]["Pclass"]))].median()
    if not np.isnan(age_pred) : # 결측값을 확인할 때.
        data['Age'].iloc[i] = age_pred
    else :
        data['Age'].iloc[i] = age_med
        
print(data['Age'].isna().sum())

In [20]:
## 모델링

## 행들이 독립적이기 때문에 kfold로 baseline에 교차검증까지 돌려서 단순 점수 확인.
## 점수 좋은 것들 중 best 모델 골라서 parameter tuning.
## tuning은 gridsearch.
## tuning 방법 중 bayesian optimization도 있고, optuna도 있다. 

## bayesian optimization
## https://www.kaggle.com/vincentlugat/ieee-lgb-bayesian-opt
## 함수를 불러와서 객체로 저장한다. 
## 후에 bound를 정하고 maximize를 한 후, 저장된 객체안에서 뽑아서 사용한다. 

## optuna
## https://www.kaggle.com/akmeghdad/tps-0921-essential
## bayesian과 같다. 

In [21]:
## 여기 모델리에서는 트리계열의 모델들이 많았는데 현재 get_dummies로 열이 67개로 불어난 상황이다.
## 특히 0,1로 이루어진 열들은 쓸데없는 가지를 치기 때문에 tree 모델이 처음에 안맞았다.

## 또한, adaboost나 gradientboosting도 모두 예전꺼에다가 계속 수정을 하며 하기 때문에 많은 estimate로 overfit 된것 같다.
## estimator를 줄이는것도 방법이 될 수 있다. 

In [22]:
## 앙상블을 할 때 결과값들의 상관관계를 보는것은 흥미로웠다.
## 후에 adaboost빼고 5개의 모델들로 hard voting해서 했다. 
## sklearn에서 찾아보면 soft도 있는데 soft는 값들의 평균. hard는 하나.

## 또 다른 방법중에는 stacking과 blending 이 있다.
## stacking은 쌓기. 모델들의 예측값들로만 또다른 데이터프레임 독립변수를 만들어서 종속변수를 예측.
## blending은 섞기. 기존 데이터와 예측값들로 다시 훈련.
## 참고 https://3months.tistory.com/486